In [1]:
import numpy as np
from matplotlib import pyplot as plt
%matplotlib inline

import seaborn as sns
sns.set_context('talk', font_scale=1.2, rc={'lines.linewidth': 3})
sns.set_style('whitegrid',
              {'grid.linestyle': ':', 'grid.color': 'red', 'axes.edgecolor': '0.5',
               'axes.linewidth': 1.2, 'legend.frameon': True})

In [2]:
import pysixtracklib as pyst
from pysixtracklib import stcommon as st

In [3]:
from cpymad.madx import Madx

In [4]:
from scipy.constants import e, m_p, c

In [5]:
import sys
sys.path = ["/home/HPC/oeftiger/PyHEADTAIL_py3/python3/PyHEADTAIL/", 
            "/home/HPC/oeftiger/PyHEADTAIL_py3/"] + sys.path

In [6]:
import time

In [7]:
from pycuda.autoinit import context
from pycuda import gpuarray as gp

In [8]:
from PyHEADTAIL.general.element import Element
from PyHEADTAIL.particles.generators import generate_Gaussian6DTwiss

from PyHEADTAIL.particles.slicing import UniformBinSlicer
from PyHEADTAIL.spacecharge.pypic_factory import create_3dmesh_from_beam
from PyHEADTAIL.spacecharge.pypic_spacecharge import (
    SpaceChargePIC, SpaceChargePIC_Adaptive25D)

from PyHEADTAIL.general.contextmanager import GPU

PyHEADTAIL v1.13.1


PyPIC v2.4.1
Info: cusolver_Rf not found. GPU finite difference solver not available.


In [9]:
from PyPIC.GPU.poisson_solver.FFT_solver import GPUFFTPoissonSolver_2_5D
from PyPIC.GPU.pypic import PyPIC_GPU

# not necessary but nice: memory pool sharing between PyHEADTAIL and PyPIC
try:
    from PyHEADTAIL.gpu.gpu_utils import memory_pool
except:
    memory_pool = None

$\implies$ for the moment for protons

In [10]:
n_macroparticles = int(1e4)
n_slices_sc = 32

# fixed field map for space charge
n_mesh_nodes = 128
n_mesh_sigma = 5

intensity = 1.3e11
epsn_x = epsn_y = 3e-6 # in [m.rad]
sigma_z = 0.23 # in [m]

p0c = 6 * 1e9 # in eV

Etot = np.sqrt(p0c**2 + (m_p/e)**2 * c**4) * 1e-9 # in GeV
p0 = p0c / c * e
gamma = np.sqrt(1 + (p0 / (m_p * c))**2)
beta = np.sqrt(1 - gamma**-2)
beta_z = 1 #np.abs(eta) * circumference / (2 * np.pi * Qs)
epsn_z = sigma_z**2 * 4 * np.pi * p0 / (e * beta_z)

In [11]:
def provide_pycuda_array(ptr):
    return gp.GPUArray(n_macroparticles, dtype=np.float64, gpudata=ptr)

# Preparing lattice

In [12]:
madx = Madx()
madx.options.echo = False


  ++++++++++++++++++++++++++++++++++++++++++++
  +     MAD-X 5.05.01  (64 bit, Linux)       +
  + Support: mad@cern.ch, http://cern.ch/mad +
  + Release   date: 2019.06.07               +
  + Execution date: 2019.06.25 14:59:10      +
  ++++++++++++++++++++++++++++++++++++++++++++


In [13]:
madx.call(file="sis18_thin.seq")

In [14]:
madx.command.beam(particle='proton', energy=str(Etot)) # energy in GeV

True

In [15]:
madx.use(sequence="FODO")

In [16]:
twiss = madx.twiss();

enter Twiss module
  
iteration:   1 error:   0.000000E+00 deltap:   0.000000E+00
orbit:   0.000000E+00  0.000000E+00  0.000000E+00  0.000000E+00  0.000000E+00  0.000000E+00

++++++ table: summ

            length             orbit5               alfa            gammatr 
       216.7081404                 -0      0.03859155171         5.09042308 

                q1                dq1            betxmax              dxmax 
       4.577598169       -4.337325087        15.52637808        2.457595991 

             dxrms             xcomax             xcorms                 q2 
       1.851915623                  0                  0        2.718441832 

               dq2            betymax              dymax              dyrms 
       -6.04407507        33.60548239                  0                  0 

            ycomax             ycorms             deltap            synch_1 
                 0                  0                  0                  0 

           synch_2            

In [17]:
circumference = twiss.summary.length
assert circumference == twiss['s'][-1]

Injecting SC markers a la Hannes:

In [18]:
n_scnodes = 5*20 #12*20
l_target = circumference / n_scnodes
l_target

2.1670814039999997

In [19]:
madx.input("""
myvalue(xx,yy,zz): macro = {myval = table(xx,yy,zz);};

sc_placeholder : Marker; 
option, -info;
l_target = """ + str(l_target) + """;
l_fuzz = l_target/2.;
rows = table(twiss,tablelength);
seqedit, sequence=FODO;
while(i<rows){
    i = i+1;
    exec, myvalue(twiss,l,$i);
    length = myval;
    if(length > l_target + l_fuzz){
        ! value, length;
        l_remaining = length;
        exec, myvalue(twiss,s,$i);
        s = myval - length;
        while (l_remaining > l_target){
            s = s + l_target;
            value, s;
            install, element=sc_placeholder, at=s;
            l_remaining = l_remaining - l_target;
        }
    }
}
flatten;
option, info;
endedit;
""")

s                  =        8.871664737 ;
s                  =        11.03874614 ;
s                  =        13.20582755 ;
s                  =        26.93067644 ;
s                  =        29.09775784 ;
s                  =        31.26483925 ;
s                  =        44.98968814 ;
s                  =        47.15676954 ;
s                  =        49.32385095 ;
s                  =        63.04869984 ;
s                  =        65.21578124 ;
s                  =        67.38286265 ;
s                  =        81.10771154 ;
s                  =        83.27479294 ;
s                  =        85.44187435 ;
s                  =        99.16672324 ;
s                  =        101.3338046 ;
s                  =         103.500886 ;
s                  =        117.2257349 ;
s                  =        119.3928163 ;
s                  =        121.5598977 ;
s                  =        135.2847466 ;
s                  =         137.451828 ;
s                  =        139.61

True

In [20]:
madx.use(sequence='FODO')

# Preparing PyHEADTAIL beam

In [21]:
D_x_0 = twiss['dx'][0] * beta
D_y_0 = twiss['dy'][0] * beta

pyht_beam = generate_Gaussian6DTwiss(
    n_macroparticles, intensity, e, m_p, circumference, gamma,
    twiss['alfx'][0], twiss['alfy'][0], twiss['betx'][0], twiss['bety'][0],
    beta_z, epsn_x, epsn_y, epsn_z, 
    dispersion_x=D_x_0 if D_x_0 else None,
    dispersion_y=D_y_0 if D_y_0 else None,
)

# Preparing PySTL for GPU

In [22]:
pyst_beam = pyst.Particles.from_ref(num_particles=n_macroparticles, p0c=p0c)

In [23]:
elements = pyst.Elements.from_mad(madx.sequence.FODO, exact_drift=True)

In [ ]:
assert (
    len(elements.get_elements()) // 2 + 1 == 
    len(madx.sequence.FODO.elements)
), ( 
    "Did not generate the same number of PySixTrackLib "
    "lattice elements as there are in the MAD-X lattice! "
    "This will mess up the computation of SC node lengths..."
)

In [ ]:
elements.BeamMonitor();

In [ ]:
trackjob = pyst.CudaTrackJob(elements, pyst_beam, until_turn_elem_by_elem=True)

In [ ]:
class TrackSixTrackLib(Element):
    def __init__(self, trackjob, i_start, i_end, context=context):
        self.trackjob = trackjob
        self.context = context
        
        self.i_start = i_start
        self.i_end = i_end
        n_elements = len(elements.get_elements())
        self.is_last_element = i_end == n_elements
        
        trackjob.fetch_particle_addresses()
        assert trackjob.last_status_success
        ptr = trackjob.get_particle_addresses() # particleset==0 is default
        
        self._x = provide_pycuda_array(ptr.contents.x)
        self._px = provide_pycuda_array(ptr.contents.px)
        self._y = provide_pycuda_array(ptr.contents.y)
        self._py = provide_pycuda_array(ptr.contents.py)
        self._z = provide_pycuda_array(ptr.contents.zeta)
        self._delta = provide_pycuda_array(ptr.contents.delta)

    def track(self, beam):
        # pass arrays and convert units
        self.pyht_to_stlib(beam)
        # track in SixTrackLib
        trackjob.track_line(self.i_start, self.i_end, 
                            finish_turn=self.is_last_element)
        self.context.synchronize()
        assert trackjob.last_track_status_success
        # pass arrays back (converting units back)
        self.stlib_to_pyht(beam)

    def pyht_to_stlib(self, beam):
        self._x[:] = beam.x
        self._px[:] = beam.xp
        self._y[:] = beam.y
        self._py[:] = beam.yp
        self._z[:] = beam.z
        self._delta[:] = beam.dp

    def stlib_to_pyht(self, beam):
        beam.x = self._x
        beam.xp = self._px
        beam.y = self._y
        beam.yp = self._py
        beam.z = self._z
        beam.dp = self._delta

# Preparing Tracking lattice

In [ ]:
idx_mad_sc = [i for i, name in enumerate(madx.sequence.FODO.element_names()) 
              if 'sc_placeholder' in name]
sc_optics = {
    'beta_x': twiss['betx'][idx_mad_sc],
    'beta_y': twiss['bety'][idx_mad_sc],
    'x': twiss['x'][idx_mad_sc],
    'y': twiss['y'][idx_mad_sc],
    's': twiss['s'][idx_mad_sc]
}

In [ ]:
sig_x = np.sqrt(sc_optics['beta_x'].max() * epsn_x / (beta * gamma))
sig_y = np.sqrt(sc_optics['beta_y'].max() * epsn_y / (beta * gamma))

In [ ]:
slicer_sc = UniformBinSlicer(n_slices_sc, n_sigma_z=4) #z_cuts=slicing_interval)

In [ ]:
mesh_3d = create_3dmesh_from_beam(pyht_beam, [n_mesh_nodes]*2, [n_mesh_sigma]*2, 
                                  slices=pyht_beam.get_slices(slicer_sc))

In [ ]:
poissonsolver = GPUFFTPoissonSolver_2_5D(mesh_3d, context=context, save_memory=False)
pypic_algorithm = PyPIC_GPU(mesh_3d, poissonsolver, context=context, 
                            memory_pool=memory_pool)

In [ ]:
circumference

216.7081404

In [ ]:
sum(el.length for el in elements.get_elements() if isinstance(el, pyst.DriftExact))

216.7081404

In [ ]:
one_turn_map = []

i_last = 1
length_covered = 0
for i_curr, el in enumerate(elements.get_elements()[:-1]):
    if not isinstance(el, pyst.DriftExact):
        continue
    length_covered += el.length
    
    #i_curr == 0 or 
    if el.length != 0: # only inject SC node at markers (for SC)
        continue
    
    pyst_node = TrackSixTrackLib(trackjob, i_last, i_curr + 1)# - 1)
    one_turn_map.append(pyst_node)
    
    sc_node = SpaceChargePIC(length_covered, pypic_algorithm)
    one_turn_map.append(sc_node)
    
    i_last = i_curr
    length_covered = 0

# assert pyst_node.i_end == len(elements.get_elements()) - 3
assert el._offset == elements.get_elements()[-2]._offset
assert isinstance(el, pyst.DriftExact)
assert el.length == 0

pyst_node.is_last_element = True

# Cross-check single-particle tracking model physics

Comparing split `TrackSixTrackLib.track` vs. `trackjob.track_elem_by_elem`

## run in one go with `track_elem_by_elem`

In [ ]:
D_x_0 = twiss['dx'][0] * beta
D_y_0 = twiss['dy'][0] * beta

np.random.seed(0)

pyht_beam = generate_Gaussian6DTwiss(
    n_macroparticles, intensity, e, m_p, circumference, gamma,
    twiss['alfx'][0], twiss['alfy'][0], twiss['betx'][0], twiss['bety'][0],
    beta_z, epsn_x, epsn_y, epsn_z, 
    dispersion_x=D_x_0 if D_x_0 else None,
    dispersion_y=D_y_0 if D_y_0 else None,
)

In [ ]:
with GPU(pyht_beam):
    pyst_node.pyht_to_stlib(pyht_beam)

In [ ]:
trackjob.track_elem_by_elem(np.max(pyst_beam.at_turn) + 1)

In [ ]:
trackjob.last_track_status_success

In [ ]:
trackjob.collect()

In [ ]:
x_pystl = trackjob.output.particles[0].x
y_pystl = trackjob.output.particles[0].y

x_pystl = x_pystl[0::n_macroparticles]#[1:][::2]
y_pystl = y_pystl[0::n_macroparticles]#[1:][::2]

In [ ]:
s_positions_all = []
for el in elements.get_elements():
    if isinstance(el, pyst.DriftExact):
        s_positions_all.append(el.length)
    else:
        s_positions_all.append(0)
s_positions_all = np.cumsum(s_positions_all)

In [ ]:
plt.plot(s_positions_all[0::2], x_pystl[0::2])
plt.twinx()
plt.grid(False)
plt.plot(s_positions_all[0::2], y_pystl[0::2], color='orange')

## run piecewise with `TrackSixTrackLib`

In [ ]:
one_turn_map_sp = [m for m in one_turn_map if isinstance(m, TrackSixTrackLib)]
s_positions = np.cumsum([0] + list(m.length for m in one_turn_map if isinstance(m, SpaceChargePIC)))

In [ ]:
D_x_0 = twiss['dx'][0] * beta
D_y_0 = twiss['dy'][0] * beta

np.random.seed(0)

pyht_beam = generate_Gaussian6DTwiss(
    n_macroparticles, intensity, e, m_p, circumference, gamma,
    twiss['alfx'][0], twiss['alfy'][0], twiss['betx'][0], twiss['bety'][0],
    beta_z, epsn_x, epsn_y, epsn_z, 
    dispersion_x=D_x_0 if D_x_0 else None,
    dispersion_y=D_y_0 if D_y_0 else None,
)

In [ ]:
with GPU(pyht_beam):
    one_turn_map_sp[0].pyht_to_stlib(pyht_beam)

In [ ]:
rec_x = np.empty(len(s_positions), dtype=float)
rec_y = np.empty_like(rec_x)

rec_x[0] = pyht_beam.x[0]
rec_y[0] = pyht_beam.y[0]

with GPU(pyht_beam):
    for i, m in enumerate(one_turn_map_sp):
        m.track(pyht_beam)
        context.synchronize()
        
        rec_x[i+1] = pyht_beam.x[0].get()
        rec_y[i+1] = pyht_beam.y[0].get()

In [ ]:
plt.plot(s_positions, rec_x)
plt.twinx()
plt.grid(False)
plt.plot(s_positions, rec_y, color='orange')

## comparison

In [ ]:
plt.figure(figsize=(15, 8))
plt.plot(s_positions_all[0::2], x_pystl[0::2], label='PySTL elem_by_elem')
plt.plot(s_positions, rec_x, '.', label='TrackSixTrackLib')
plt.legend();

# Timing

In [ ]:
t0 = time.time()

for m in one_turn_map:
    m.track(pyht_beam)
    
t1 = time.time()

In [ ]:
t1 - t0

In [ ]:
stats = %prun -r -q   for m in one_turn_map: m.track(pyht_beam)

In [ ]:
stats.sort_stats(2).print_stats(20)

In [ ]:
stats = %prun -r -q for m in one_turn_map_sp: m.track(pyht_beam)

In [ ]:
stats.sort_stats(2).print_stats(20)